In [151]:
import string # for lowercase letter list
import numpy as np

In [152]:
with open('/usr/share/dict/words') as o:
    DICTIONARY = set([s.strip() for s in o.readlines() if not s[0].isupper()])

In [153]:
LETTERS =  string.ascii_lowercase

In [154]:
class RandomHangManStrategy:
 
    def init(self):
        np.random.seed(1)
    
    def guess(self, word, excluded_letters, possibilities):
        possible_letter = [letter for letter in LETTERS if not letter in excluded_letters]
        return np.random.choice(possible_letter, 1)[0]

In [155]:
class SimpleHangManStrategy:
    
    def init(self):
        pass
    
    def guess(self, word, excluded_letters, possibilities):
        letter_counts = {}
        for word in possibilities:
            for letter in word:
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1
        
        most_common_letter, max_count = '?', 0
        for letter, count in letter_counts.items():
            if count > max_count and letter not in excluded_letters:
                max_count = count
                most_common_letter = letter
                
        return most_common_letter

In [171]:
class VowelHangManStrategy:
    
    def __init__(self, bias=1):
        self.common_letters = ['a', 'e', 'i', 'o', 'u', 'y']
        self.bias = bias
    
    def init(self):
        pass
    
    def guess(self, word, excluded_letters, possibilities):
        letter_counts = {}
        for word in possibilities:
            for letter in word:
                if letter in self.common_letters:
                    if letter in letter_counts:
                        letter_counts[letter] += 1 * self.bias
                    else:
                        letter_counts[letter] = 1 * self.bias
                else:
                    if letter in letter_counts:
                        letter_counts[letter] += 1
                    else:
                        letter_counts[letter] = 1
        
        most_common_letter, max_count = '?', 0
        for letter, count in letter_counts.items():
            if count > max_count and letter not in excluded_letters:
                max_count = count
                most_common_letter = letter
                
        return most_common_letter

In [172]:
class HangManPlayer:
    
    def start(self, word_length):
        self.excluded_letters = []
        self.word = '?' * word_length
        self.possibilities = sorted([word for word in DICTIONARY if len(word) == word_length])
        self.guesses = 0
        self._strategy.init()
    
    def __init__(self, name, strategy):
        self.name = name
        self._strategy = strategy
        
    def guess(self):
        if '?' not in self.word:
            return f'The word is {self.word}. Guessed in {self.guesses} tries'
        
        if len(self.possibilities) == 0:
            return f'No possible words to try: {self.word}' 
        
        self.guesses += 1
        return self._strategy.guess(self.word, self.excluded_letters, self.possibilities)
    
    def finished(self):
        return '?' not in self.word
    
    @staticmethod
    def _keep_word(word, letter, positions):
        for i in range(len(word)):
            if word[i] != letter and i in positions:
                return False
            elif word[i] == letter and i not in positions:
                return False
        return True
    
    def update_word(self, letter, positions=None):
        positions = positions or []
        self.excluded_letters.append(letter)
 
        # Update word
        updated_word = [letter for letter in self.word]
        for position in positions:
            if updated_word[position] != '?':
                raise Exception(f'Letter at position {position} is already specified as {updated_word[position]}')
            updated_word[position] = letter
        self.word = ''.join(updated_word)

        # Update possibilities
        self.possibilities = [
            word for word in self.possibilities 
            if HangManPlayer._keep_word(word, letter, positions)
        ]

In [173]:
class HangManGame:
    
    def __init__(self, word, player):
        self.word = word
        self.player = player
        self.steps = 0
        
    def finished(self):
        return self.player.finished()
        
    def step(self):
        assert not self.finished(), "Game is finished!"
        self.steps += 1
        guess = self.player.guess()
        positions = []
        for i, letter in enumerate(self.word):
            if letter == guess:
                positions.append(i)
        self.player.update_word(guess, positions)
        if positions:
            print(f"{self.player.name} correctly guessed '{guess}'.")
        else:
            print(f"{self.player.name} incorrectly guessed '{guess}'.")

In [174]:
def run_game(word, player):
    game = HangManGame(word, player)
    player.start(len(word))

    while not game.finished():
        game.step()
    print(f"{player.name} got '{word}' in {game.steps}.")

In [175]:
def compare_players(word, players):
    for player in players:
        run_game(word, player) 

In [178]:
word = "practical"
players = [
    HangManPlayer("Simple", SimpleHangManStrategy()),
    HangManPlayer("Vowel-focused", VowelHangManStrategy(bias=1)),
    HangManPlayer("Random", RandomHangManStrategy())
]
compare_players(word, players)

Simple incorrectly guessed 'e'.
Simple correctly guessed 'i'.
Simple correctly guessed 'a'.
Simple correctly guessed 'l'.
Simple correctly guessed 'c'.
Simple correctly guessed 'p'.
Simple correctly guessed 'r'.
Simple correctly guessed 't'.
Simple got 'practical' in 8.
Vowel-focused incorrectly guessed 'e'.
Vowel-focused correctly guessed 'i'.
Vowel-focused correctly guessed 'a'.
Vowel-focused correctly guessed 'l'.
Vowel-focused correctly guessed 'c'.
Vowel-focused correctly guessed 'p'.
Vowel-focused correctly guessed 'r'.
Vowel-focused correctly guessed 't'.
Vowel-focused got 'practical' in 8.
Random incorrectly guessed 'f'.
Random incorrectly guessed 'm'.
Random incorrectly guessed 'o'.
Random incorrectly guessed 'j'.
Random correctly guessed 'l'.
Random incorrectly guessed 'q'.
Random incorrectly guessed 'g'.
Random incorrectly guessed 'w'.
Random correctly guessed 'a'.
Random incorrectly guessed 'z'.
Random correctly guessed 'c'.
Random incorrectly guessed 'v'.
Random correctly 